# Adding Dependeces

In [26]:
using DelimitedFiles
using CUDA
using ProgressMeter
include("../../src/forces.jl")
include("../../src/Neighbor.jl")

knn_cu (generic function with 1 method)

# Initial Conditions

In [27]:
# Running fusion of the two aggregates
# Physical Conditions
R_agg = 10

# Time model Conditions
T_f = 150000
dt = 0.1

# Neighbor Conditions
n_knn = 50
nn = 16

# Forces Conditions
R_Max = [3.2 3.5]
S = 2.0
K_array = [1 0.5 0.1 0.05 0.01 0.005]

# Coding C0nditions
n_text = 50

50

# Running Fusion

In [28]:
# Testing all the conditions of the fusion
for t_f in T_f
    for r_max in R_Max
        for s in S
            for K in K_array
                # Initial Data
                X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu
                
                #Generating the path
                if !isdir("../../results/Fusion/T_$(t_f)") mkdir("../../results/Fusion/T_$(t_f)") end
                if !isdir("../../results/Fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)") mkdir("../../results/Fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)") end
                # if !isdir("../../results/Fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)") mkdir("../../results/Fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)/k_$(K)")  end
                
                # # Calculating all above
                Folder = readdir("../../results/Fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)")
                File = "tf_($(t_f))|dt_($(dt))|rm_($(r_max))|s=($(s))|K_($(K))_GPU.xyz"
                println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | s = $(s) | k = $(K)")
                if !(File in Folder)
                    fusion("../../results/Fusion/T_$(t_f)/rmax_$(r_max)_s_$(s)/",n_text,t_f, r_max, s, K)
                else
                    println("Calculating T_Final=$(t_f) | R_Max = $(r_max) | s = $(s) | k = $(K)")
                    println("This is already calculated")
                end
            end
        end
    end
end

Calculating T_Final=150000 | R_Max = 3.2 | s = 2.0 | k = 1.0


Progress: 100%|█████████████████████████| Time: 0:15:02


Calculating T_Final=150000 | R_Max = 3.2 | s = 2.0 | k = 0.5


Progress: 100%|█████████████████████████| Time: 0:14:33


Calculating T_Final=150000 | R_Max = 3.2 | s = 2.0 | k = 0.1


...

Progress: 100%|█████████████████████████| Time: 0:13:31


Calculating T_Final=150000 | R_Max = 3.5 | s = 2.0 | k = 0.01


Progress: 100%|█████████████████████████| Time: 0:13:43


Calculating T_Final=150000 | R_Max = 3.5 | s = 2.0 | k = 0.005


Progress: 100%|█████████████████████████| Time: 0:13:53
